# Pendulum ODE with odeint and Runge-Kutta


In [ ]:
#theta''(t) + b*theta'(t) + c*sin(theta(t)) = 0
#theta'(t) = omega(t)
#omega'(t) = -b*omega(t) - c*sin(theta(t))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
pip install scipy

In [ ]:
from scipy.integrate import odeint  # for comparison

In [ ]:
def pend(y, t, b, c):
    theta, omega = y
    dydt = [omega, -b*omega - c*np.sin(theta)]
    #dydt = [- np.sin(omega),- np.sin(theta)]
    return dydt

In [ ]:
b = 0.25
c = 5.0

In [ ]:
y0 = [np.pi - 0.1, 0.0]

In [ ]:
t = np.linspace(0, 10, 101)

In [ ]:
sol = odeint(pend, y0, t, args=(b, c))

In [ ]:
np.shape(sol)

In [ ]:
plt.plot(t, sol[:, 0], 'b', label=r'$\theta(t)$')
plt.plot(t, sol[:, 1], 'g', label=r'$\omega(t)$')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
#runge kutta
def rungekutta1(f, y0, t, args=()):
    n = len(t)
    y = np.zeros((n, len(y0)))
    y[0] = y0
    for i in range(n - 1):
        y[i+1] = y[i] + (t[i+1] - t[i])*np.array(f(y[i], t[i], *args))
        #print(type(t[i]))
        #print (np.array(f(y[i], t[i], *args)))
    return y

In [ ]:
sol = rungekutta1(pend, y0, t, args=(b, c))

In [ ]:
plt.plot(t,sol)

In [ ]:
#y'=cos t
def sin(y, t):
    dydt = np.cos(t)
    return dydt

In [ ]:
sol2 = odeint(sin, 0, t)

In [ ]:
plt.plot(t, sol2, 'b', label=r'$\theta(t)$')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
#runge kutta
def rungekutta1(f, y0, t, args=()):
    n = len(t)
    y = np.zeros(n)
    y[0] = y0
    for i in range(n - 1):
        y[i+1] = y[i] + (t[i+1] - t[i]) * f(y[i], t[i])
    return y

In [ ]:
y0 = 0

In [ ]:
# choose numsteps
numsteps = 10
t = np.linspace(0,10,numsteps)

In [ ]:
sol = rungekutta1(sin, y0, t)

In [ ]:
plt.plot(t, sol, 'm', label=r'$\theta(t)$')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

## Runge-Kutta for multiple initial conditions

In [ ]:
# Let us improve runge kutta
def rungekutta_mult(f, y0, t, args=()):
    n = len(t)
    y = np.zeros((len(y0), n))
    y[:, 0] = y0
    for i in range(n - 1):
        y[:, i+1] = y[:, i] + (t[i+1] - t[i]) * f(y[:, i], t[i])
    return y

def solve( y0 ):
  y = np.zeros( (len(y0), N))
  y[:,0] = y0
  for i in range(1,N):
    t_i = i*dt
    dy = dt*y_prime( t_i, y[:,i-1] )
    y[:, i] = y[:, i-1] + dy
  #
  return y

In [ ]:
y0 = np.random.rand(10)

In [ ]:
solutions = rungekutta_mult(sin, y0, t)
#solutions = rungekutta1(sin, y0, t)

In [ ]:
import matplotlib.pyplot as plt

plt.plot( solutions.T )
plt.title( "Curves solutions to the ODE")
plt.xlabel( "Time t")

In [ ]:
#Next thing is to try runge kutta for geodesics ode
#Let us first try to compute geodesics on a sphere or any simpe 2-manifold
import numpy as np
u = np.zeros(2)
phi, theta = u

In [ ]:
#Christoffel symbols
#u = np.zeros(2)
R = 3
def g (u): #metrics on sphere
    phi, theta = u
    g = (R**2)*np.array([[np.cos(theta)**2, 0],[0, 1]])
    return g

In [ ]:
# check
g = g([0,np.pi/4])

In [ ]:
g

In [ ]:
def g_inv (u): #inverse metrics on sphere
    phi, theta = u
    g_inv = (1/R**2)*np.array([[1/np.cos(theta)**2, 0],[0, 1]])
    return g_inv

In [ ]:
# check
g_inv([0,np.pi/4])

In [ ]:
# alternative way to inverse matrix via linalg
h = np.linalg.inv(g)

In [ ]:
h

In [ ]:
#derivatives of metrics on sphere
def dgdphi (u): #dg/dphi
    phi, theta = u
    g = np.array([[0, 0],[0, 0]])
    return g
def dgdtheta (u): #dg/dtheta
    phi, theta = u
    g = np.array([[-R**2*np.sin(2*theta), 0],[0, 0]])
    return g

In [ ]:
def dg (u): #dg
    phi, theta = u
    g = np.array([[[0, 0],
                   [0, 0]],
                  [[-R**2*np.sin(2*theta), 0],
                   [0, 0]]])
    return g

In [ ]:
dgdtheta([0,np.pi/4])

In [ ]:
dg([0,np.pi/4])[1,0,0] #first index is the index of the variable wrt which we differentiate

In [ ]:
#Christoffel symbols at a point u = phi , theta
def Ch(u):
    Ch = np.zeros((2,2,2))
    for i in range(2):
        for j in range(2):
            for l in range(2):
                for k in range(2):
                    Ch[l,i,j] += 0.5 * g_inv(u)[l,k] * (dg(u)[i,k,j] + dg(u)[j,i,k] - dg(u)[k,i,j]) #Ch^l_ij
    return Ch

In [ ]:
Ch([0,np.pi/4])

In [ ]:
#computing geodesics...
# y = [u , v]
# v := dot(u)
# dot(v)^l = Ch^l_ij * v^i * v^j
def geod(y, t):
    #u, v = y
    u = y[0:2:]
    v = y[2::]
    dudt = v
    dvdt = np.zeros(2)
    for l in range(2):
        for i in range(2):
            for j in range(2):
                dvdt[l] -= Ch(u)[l,i,j] * v[i] * v[j]
    dydt = np.concatenate((dudt, dvdt))
    return dydt

In [ ]:
#\phi is the andle to some fixed meridin (longitude), \theta is the angle to the equator (lattitude)
# u = \phi, \theta
u0 = [0.1, 0.0] # initial position in local coord \phi, \theta
v0 = [0.0, 1.0] # initial speed
y0 = np.concatenate((u0,v0))

In [ ]:
y0

In [ ]:
t = np.linspace(0, 2, 21)

In [ ]:
sol = odeint(geod, y0, t)

In [ ]:
np.shape(sol)

In [ ]:
sol

In [ ]:
#draw the graphs of solutions phi and theta
plt.plot(t, sol[:, 0], 'b', label=r'$\phi(t)$')
plt.plot(t, sol[:, 1], 'g', label=r'$\theta(t)$')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()
#this should be like a meridian

In [ ]:
plt.plot (sol[:, 0], sol[:, 1])
plt.grid()
plt.show()

In [ ]:
#runge kutta
def rungekutta1(f, y0, t, args=()):
    n = len(t)
    y = np.zeros((n, len(y0)))
    y[0] = y0
    for i in range(n - 1):
        y[i+1] = y[i] + (t[i+1] - t[i])*f(y[i], t[i], *args)
        print(y[i])
    return y
sol = rungekutta1(geod, y0, t)

## Vectorization of Christoffel symbols and metric derivatives

In [ ]:
# let us compute everithing on a grid. i.e. first dimension would give us the index of the node on the vectorized grid
import torch
def g (u): #metrics on a grid
    # u is the vector of points
    R = 3 #Radius
    phi = u[:,0]
    theta = u[:, 1]
    n = u.shape[0] #number of points
    g = torch.zeros((n,2,2))
    #g11 = torch.cos(theta)**2
    #g12 = torch.zeros(n)
    #g21 = torch.zeros(n)
    #g22 = torch.ones(n)

    #hyperbolic metric on a half plane
    g11 = 1/theta**2
    g12 = torch.zeros(n)
    g21 = torch.zeros(n)
    g22 = 1/theta**2

    g = torch.cat((g11, g12, g21, g22)).view(4,n)
    g = g.T
    g = g.view(n, 2, 2)
    #g = (R**2)*g
    #g = (R**2)*torch.tensor([[torch.cos(theta)**2, 0],[0, 1]])
    return g

In [ ]:
g(torch.tensor([[0.,0.1],[0.,0.1],[0.,1.]]))

In [ ]:
u = torch.tensor([[0.,0.],[0.,0.],[0.,1.]])

In [ ]:
theta = u[:, 1]
theta

In [ ]:
1/theta**2

In [ ]:
numsteps = 3
xs = torch.linspace(-1, 1, steps = numsteps)
ys = torch.linspace(-1, 1, steps = numsteps)
grid = torch.cartesian_prod(xs,ys)

In [ ]:
grid.shape

In [ ]:
yy = g(grid)

In [ ]:
yy

In [ ]:
torch.cat((yy,yy),1).view(9,2,2,2)

In [ ]:
torch.inverse(yy) # inverts several matrices at once!!

In [ ]:
# metric derivatives on a grid for hyperbolic metric
def dg (u): #dg
    #phi, theta = u
    #think of x = phi, y = theta
    # u is the vector of points
    R = 3 #Radius
    phi = u[:,0]
    theta = u[:, 1]
    n = u.shape[0] #number of points
    g = torch.zeros((n,2,2,2))
    
    #x derivatives of g
 
    gx11 = torch.zeros(n)
    gx12 = torch.zeros(n)
    gx21 = torch.zeros(n)
    gx22 = torch.zeros(n)

    gx = torch.cat((gx11, gx12, gx21, gx22)).view(4,n)
    gx = gx.T
    gx = gx.view(n, 2, 2)
    
    #y derivatives of g
    
    #gy11 = -R**2*torch.sin(2*theta)
    gy11 = -2/theta**3
    gy12 = torch.zeros(n)
    gy21 = torch.zeros(n)
    gy22 = -2/theta**3
    #gy22 = torch.zeros(n)

    gy = torch.cat((gy11, gy12, gy21, gy22)).view(4,n)
    gy = gy.T
    gy = gy.view(n, 2, 2)

    dg = torch.cat((gx,gy),1).view(n,2,2,2)
    #g = np.array([[[0, 0],
    #               [0, 0]],
    #              [[-R**2*np.sin(2*theta), 0],
    #               [0, 0]]])
    return dg

In [ ]:
dg(grid[:3])

In [ ]:
#Christoffel symbols at a vector of points  u = num of points, phi , theta
#this code does not use loops
def Ch(u):
    #phi = u[:,0]
    #theta = u[:, 1]
    n = u.shape[0]
    Ch = torch.zeros((n, 2,2,2))
    for i in range(2):
        for j in range(2):
            for l in range(2):
                for k in range(2):
                    Ch[:,l,i,j] += 0.5 * torch.inverse(g(u))[:,l,k] * (dg(u)[:,i,k,j] + dg(u)[:,j,i,k] - dg(u)[:,k,i,j]) #Ch^l_ij
                    #Ch[l,i,j] += 0.5 * g_inv(u)[l,k] * (dg(u)[i,k,j] + dg(u)[j,i,k] - dg(u)[k,i,j]) #Ch^l_ij
    return Ch

In [ ]:
Ch(grid)

In [ ]:
Ch(torch.tensor([[0.,0.]]))

In [ ]:
u0 = torch.tensor([0.0, 0.1]) # initial position in local coord \phi, \theta
v0 = torch.tensor([1.0, 0.0]) # initial speed
y0 = torch.cat((u0,v0)).view(1,4)
t = torch.linspace(0, 2, steps = 21)

In [ ]:
y0.shape

In [ ]:
t = torch.linspace(0, 2, steps = 21)

In [ ]:
#computing geodesics...
# y has shape num of points, u, v
# v := dot(u)
# dot(v)^l = Ch^l_ij * v^i * v^j
def geod(y, t):
    #u, v = y
    n = y.shape[0]
    u = y[: , 0:2:]
    v = y[: , 2::]
    dudt = v
    dvdt = torch.zeros(n, 2)
    for l in range(2):
        for i in range(2):
            for j in range(2):
                dvdt[:, l] -= Ch(u)[:, l,i,j] * v[:, i] * v[:, j]
    dydt = torch.cat((dudt.T, dvdt.T)).T
    # dydt = np.concatenate((dudt, dvdt))
    return dydt

In [ ]:
geod(y0,t)

In [ ]:
h = torch.rand(10,4)

In [ ]:
geod(h,t)

In [ ]:
#runge kutta for many initial conditions
def rungekutta_new(f, y0, t, args=()):
    nt = len(t) # number of steps in time
    # len(y0[0]) is the number of initial conditions
    # len(y0[1]) is the dimention of the state space. In our case it is 4 
    y = torch.zeros((nt, y0.shape[0],y0.shape[1]))
    y[0,:,:] = y0
    for i in range(nt - 1):
        y[i+1,:,:] = y[i,:,:] + (t[i+1] - t[i])*f(y[i,:,:], t[i], *args)
        print(y[i,:,:])
    return y

In [ ]:
sol0 = rungekutta_new(geod, y0, t)
#solrand = rungekutta_new(geod, h, t)

In [ ]:

import matplotlib.pyplot as plt

#plt.plot(t, solrand[:, :, 0], 'b', label=r'$\phi(t)$')
plt.plot(sol0[:, :, 0], sol0[:, :, 1], 'g', label=r'$\theta(t)$')
#plt.plot(t, solrand[:, :, 1], 'g', label=r'$\theta(t)$')
#plt.legend(loc='best')
#plt.plot( sol.T )
plt.title( "Parametric plots of the coordinates of geodesics")
plt.xlabel( "Time t")
plt.grid()

In [ ]:
plt.plot(solrand[:, :, 0], solrand[:, :, 1])
#plt.plot(t, solrand[:, :, 1], 'g', label=r'$\theta(t)$')
#plt.legend(loc='best')
#plt.plot( sol.T )
plt.title( "Plots of geodesics with random initial conditions")
plt.xlabel( "x coordinate")
plt.ylabel( "y coordinate")
plt.grid()